In [31]:
from mysql.connector import MySQLConnection, Error
import pandas as pd

In [33]:
def make_connection():
    """
    Make a connection to the school database.
    """
    try:
        conn = MySQLConnection(host = 'IES-ADS-ClassDB.sjsu.edu',
                               database = 'querycrew_wh',
                               user = 'querycrew_user',
                               password = 'Pomegranate_746')
        
        if conn.is_connected():
            print('Connected to the MySQL database!')
            
            return conn
                
    except Error as e:
        print('Connection failed.')
        print(e)
        
        return None

In [35]:
conn = make_connection()
cursor = conn.cursor()

Connected to the MySQL database!


This query finds the number of EV arrivals per manufacturer using the ev_arrivals table

In [38]:
query = (
    """
    SELECT manufacturer_id, COUNT(arrival_id) AS number_of_arrivals
    FROM EV_arrivals
    GROUP BY manufacturer_id
    ORDER BY manufacturer_id
    LIMIT 20;
    """
)

cursor.execute(query)
results = cursor.fetchall()

# cursor description to get metdata about the columns which we are selecting
column_names = [i[0] for i in cursor.description]

df = pd.DataFrame(results, columns=column_names)

print(df)

   manufacturer_id  number_of_arrivals
0              103                   1
1              106                   2
2              107                   2
3              110                   1
4              112                   1
5              113                   1
6              114                   1
7              116                   1
8              117                   1
9              127                   1
10             128                   1
11             143                   1
12             146                   1
13             150                   1
14             156                   1
15             157                   1
16             163                   1
17             165                   1
18             180                   1
19             187                   1


This query finds the latest arrival for each dealer and displays the make and model of the ev

In [41]:
query = (
    """
    SELECT dealer_id, MAX(arrival_date) AS latest_arrival_date, ev_make, ev_model
    FROM EV_arrivals
    WHERE arrival_date IN (SELECT MAX(arrival_date) from EV_arrivals GROUP BY dealer_id)
    GROUP BY dealer_id, ev_make, ev_model
    ORDER BY dealer_id;
    """
)

cursor.execute(query)
results = cursor.fetchall()

# cursor description to get metdata about the columns which we are selecting
column_names = [i[0] for i in cursor.description]

df = pd.DataFrame(results, columns=column_names)

print(df)

   dealer_id latest_arrival_date  ev_make         ev_model
0          1          2020-12-04     Ford  F-150 Lightning
1         10          2023-02-03      Kia              Ev6
2         11          2021-09-27     Audi        Q4 E-Tron
3         12          2022-07-13      Kia             Niro
4         13          2023-12-20      Bmw               Ix
5         14          2025-09-22  Hyundai          Ioniq 6
6         15          2022-08-28      Bmw               I3
7         16          2022-04-04      Bmw               I3
8         17          2021-10-16  Hyundai             Kona
9         18          2025-06-20  Hyundai          Ioniq 5
10        19          2025-12-21   Toyota      Prius Prime
11         2          2025-04-21    Honda          Clarity
12        20          2023-06-21      Kia             Niro
13         3          2025-10-06   Nissan            Ariya
14         4          2020-09-24      Kia              Ev6
15         5          2024-08-16      Bmw               

This query finds the total sales revenue per car make and car model:

In [44]:
query = (
    """
    SELECT car_make as make, car_model as model, SUM(price) AS total_revenue
    FROM car_sales
    GROUP BY car_make, car_model
    ORDER BY car_make, car_model;
    """
)

cursor.execute(query)
results = cursor.fetchall()

# cursor description to get metdata about the columns which we are selecting
column_names = [i[0] for i in cursor.description]

df = pd.DataFrame(results, columns=column_names)

print(df)

       make        model  total_revenue
0      Audi           A4        28025.0
1      Audi           A6        32365.0
2      Audi           Q5        82754.0
3      Audi           Q7        78686.0
4       Bmw     3 Series       143202.0
5       Bmw     5 Series        84023.0
6       Bmw           X3       215256.0
7     Chevy      Equinox        54261.0
8     Chevy       Impala        37355.0
9     Chevy       Malibu        57120.0
10    Chevy  Trailblazer       189366.0
11     Ford       Escape        70474.0
12     Ford     Explorer        34042.0
13     Ford        Focus       125112.0
14     Ford       Fusion        27587.0
15    Honda       Accord        44903.0
16    Honda        Civic       229630.0
17    Honda         Cr-V       185081.0
18  Hyundai      Elantra        75007.0
19  Hyundai     Santa Fe        89849.0
20  Hyundai       Sonata        76771.0
21  Hyundai       Tucson       142781.0
22      Kia       Optima        79566.0
23      Kia      Sorento        57588.0


This query finds the average price of cars sold by each dealer

In [47]:
query = (
    """
    SELECT dealer_id, ROUND(AVG(price),2) AS average_price
    FROM car_sales
    GROUP BY dealer_id
    ORDER BY dealer_id
    LIMIT 20;
    """
)

cursor.execute(query)
results = cursor.fetchall()

# cursor description to get metdata about the columns which we are selecting
column_names = [i[0] for i in cursor.description]

df = pd.DataFrame(results, columns=column_names)

print(df)

   dealer_id  average_price
0         10       16578.00
1        100       17557.00
2        103       34607.00
3        106       16745.00
4        107       29769.00
5        108       28275.67
6        109       21608.00
7         11       25626.00
8        111       31040.00
9        115       28148.50
10       116       24684.00
11       117       19409.00
12       118       29425.00
13       121       21865.00
14       122       37246.00
15       127       24100.00
16       129       33697.00
17        13       28025.00
18       130       28533.00
19       135       32660.50
